# Genetic Algorithm

Natural Selection based.

Initial Population -> Fitness Function -> Selection -> Crossover -> Mutation 

https://levelup.gitconnected.com/how-to-implement-a-traveling-salesman-problem-genetic-algorithm-in-python-ea32c7bef20f

In [4]:
import random

In [6]:
cities = [
    'F',  # "Fortaleza",
    'C',  # "Caucaia",
    'J',  # "Juazeiro do Norte",
    'M',  # "Maracanaú"
    'S'  # "SOBRAL"
]
costs = [
    [0, 10, 100, 50, 23],
    [10, 0, 20, 20, 12],
    [100, 20, 0, 10, 50],
    [50, 20, 10, 0, 17],
    [23, 12, 50, 17, 0]
]

# Idea: F -> C -> J -> M -> F

In [7]:
# Since we will always be back to initial state, doesn't seem smart to set start state inside the DNA  
bases = "1234"  # == C J M

In [8]:
gene_example = "1342"  # = CMJ
best_gene = "123"  # ?

In [11]:
def fitness(gene: str) -> int:
    cost = 0
    prev = 0
    for dna in gene:
        n = int(dna)
        cost += costs[prev][n]
        prev = dna

    return cost

In [12]:
random_gen = random.Random(1)


def genesis(n: int) -> list[str]:
    pop = []
    for i in range(n):
        dna = ''
        for _ in range(len(bases)):
            choice = random_gen.choice(bases)
            dna += choice
        pop.append(dna)

    return pop

In [15]:
def progenitor_selection(population_set: list[str], choice_p=.2, size=.5) -> list[tuple[str, str]]:
    progenitors = []
    valuable_progenitors = []
    for gene in population_set:
        v = fitness(gene)
        valuable_progenitors.append((gene, v))

    sorted_values = sorted(valuable_progenitors, key=lambda x: x[1])

    random_gen.randint(0, len(sorted_values) - 1)
    for gene, fit in sorted_values:
        pass

    return progenitors